[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1YnDHZY6owEypgSgvJJgXPcntxOjy9GpS?usp=sharing)


# Build a Chatbot with Guardrails: Using Langchain and Portkey to Enforce PII Detection and more

In this tutorial, we'll create a customer support chatbot using LangChain and Portkey guardrails to protect sensitive information, focusing on preventing PII exposure in logs.


Portkey's **Guardrails** offer real-time enforcement of LLM behavior, with features including:

- **Regex matching** and **JSON Schema validation**
- **Code detection**
- **Custom guardrail**s: Integrate your existing guardrail systems custom guardrail integration
-** LLM-based guardrails** (e.g., gibberish detection, prompt injection scanning)

With **20+ deterministic guardrail**s and integrations with platforms like **Aporia, Pillar and Patronus AI,** Portkey provides comprehensive AI safety solutions. Guardrails can be configured for inputs, outputs, or both, with actions ranging from request denial to alternative LLM fallbacks.

For more details, visit the [Portkey Guardrails documentation](https://docs.portkey.ai/docs/product/guardrails/list-of-guardrail-checks/patronus-ai).


Let's build our guardrail-protected chatbot!

# Setting Up Portkey Guardrails

Before building our chatbot, let's configure Portkey guardrails for PII detection:

1. Sign Up for [portkey.ai](https://portkey.ai) (the dev account is free ferver) 🎊
2. Enable Patronus AI in the [Integrations page](https://app.portkey.ai/integrations) on Portkey App
3. Navigate to the [Guardrails dashboard](https://app.portkey.ai/guardrails)
4. Create a new guardrail, by selecting "Detect PII Guardrail" by Patronus AI
6. Save and copy your Config ID



This Config ID is crucial for integrating PII protection into our LangChain-Portkey  chatbot. With this setup, we're ready to create a secure, PII-aware conversational AI.


![O](https://raw.githubusercontent.com/siddharthsambharia-portkey/Portkey-Product-Images/main/Portkey_PII_Guardrails.png)



# Step 1: Setting up the environment

First, let's install the necessary packages:

In [ ]:
!pip install -qU langchain-openai portkey-ai langchain langchain_community

# Step 2: Importing required libraries and setting up API keys

In this step, we'll import the necessary libraries and set up our API keys. We'll also create Portkey config object to add guardrails to protect against PII exposure. Get your Portkey API key from [here](https://app.portkey.ai/api-keys).

In [ ]:
import os
from typing import Dict
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from portkey_ai import createHeaders, PORTKEY_GATEWAY_URL
from langchain_core.chat_history import InMemoryChatMessageHistory


portkey_config={
	"before_request_hooks": [{
		"id": "YOUR_CONFIG_ID" #Enter the config ID you saved from portkey dashboard
	}],
	"after_request_hooks": [{
		"id": "YOUR_CONFIG_ID" #Enter the config ID you saved from portkey dashboard
	}]
}


model = ChatOpenAI(
    api_key="YOUR_OPENAI_API_KEY", #Enter your OpenAI API key
    base_url=PORTKEY_GATEWAY_URL,
    model="gpt-3.5-turbo",
    default_headers=createHeaders(
        provider="openai",
        api_key="YOUR_PORTKEY_API_KEY" #Enter your Portkey API key
        config=portkey_config,
    )
)


This code sets up our environment with the necessary configurations for Portkey guardrails. The portkey_config defines our guardrail settings, including retry attempts, caching, and hooks for request and response processing.



# Step 3: Creating the chat prompt template and chain

Now, let's set up our chat prompt template and create a chain that combines the prompt with our Portkey-protected model. This structure allows us to maintain a conversation history while applying guardrails to each interaction.

In [ ]:

# Create a chat prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
    MessagesPlaceholder(variable_name="messages"),
])

# Create a chain that combines the prompt and the model
chain = prompt | model

# Create a dictionary to store chat histories for different sessions
store: Dict[str, BaseChatMessageHistory] = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Wrap the chain with message history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

# Function to chat with the bot
def chat_with_bot(session_id: str, user_input: str):
    config = {"configurable": {"session_id": session_id}}
    response = with_message_history.invoke(
        {"messages": [{"content": user_input, "type": "human"}]},
        config=config
    )
    return response.content


This code creates a chat prompt template, sets up a chain combining the prompt and the Portkey-protected model, and establishes a system for managing chat history. The chat_with_bot function handles individual interactions, applying our guardrails to each message.


# Step 4: Running the chatbot
Finally, we'll implement the main loop for our chatbot. This will allow users to interact with the bot while our Portkey guardrails work in the background to protect sensitive information.

Try using this prompt to see guardrails in action- "My name is Siddharth and I am working at Portkey AI"

In [ ]:

# Example usage
if __name__ == "__main__":
    session_id = "abc123"

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Bot: Goodbye!")
            break

        response = chat_with_bot(session_id, user_input)
        print(f"Bot: {response}")

You: hey
Bot: Hello! How can I help you today?
You: my name is siddharth and I am working at Portkey AI


APIStatusError: Error code: 446 - {'error': {'message': 'The guardrail checks defined in the config failed. You can find more information in the `hook_results` object.', 'type': 'hooks_failed', 'param': None, 'code': None}, 'hook_results': {'before_request_hooks': [{'verdict': True, 'id': 'pg-detect-8c7f58', 'checks': [{'error': {'message': 'Patronus guardrails only support after_request_hooks.'}, 'verdict': True, 'data': None, 'id': 'patronus.pii', 'execution_time': 0, 'created_at': '2024-09-03T14:16:59.363Z'}], 'feedback': {'value': 5, 'weight': 1, 'metadata': {'successfulChecks': 'patronus.pii', 'failedChecks': '', 'erroredChecks': ''}}, 'execution_time': 0, 'async': False, 'type': 'guardrail', 'created_at': '2024-09-03T14:16:59.363Z', 'deny': False}], 'after_request_hooks': [{'verdict': False, 'id': 'pg-pirate-30748a', 'checks': [{'verdict': True, 'id': 'default.contains', 'execution_time': 0, 'created_at': '2024-09-03T14:17:00.313Z'}, {'verdict': False, 'data': {'score_raw': 0, 'p

## Conclusion

Integrating Portkey with our LangChain-based chatbot has significantly enhanced our AI application's capabilities, reliability, and observability. Portkey offers a comprehensive suite of tools designed to streamline AI development and operations:

### AI Gateway
- Fast, multimodal gateway supporting multiple LLM providers
- Universal API for easy integration
- Features like caching, fallbacks, and canary testing
- Secure vault for API key management

### Reliability
- Automatic retries and load balancing
- Fallback options and custom timeout management
- Caching to reduce costs and latency

### Observability
- OpenTelemetry-compliant suite for comprehensive insights
- Detailed logging and request tracing
- Analytics dashboard with 21+ key metrics
- Customizable filters and metadata tagging

By leveraging these features, we've created a more robust, efficient, and transparent AI application. Portkey's tools not only enhance our chatbot's performance and stability but also provide crucial insights for continuous improvement. As we scale our AI systems, these capabilities will be instrumental in maintaining high-quality service, optimizing resource utilization, and driving data-driven decisions in our AI initiatives.